In [1]:
import pandas as pd
import numpy as np
import keras
import nltk

Using TensorFlow backend.


In [2]:
train=pd.read_csv(r'C:\Users\neera\OneDrive\Desktop\Projects\Aspect Based Sentiment Analysis\Sentiment analysis\imbd-movie-reviews-for-binary-sentiment-analysis\MovieReviewTrainingDatabase.csv')

In [3]:
train.head()

,sentiment,review
0,Positive,With all this stuff going down at the moment w...
1,Positive,'The Classic War of the Worlds' by Timothy Hin...
2,Negative,The film starts with a manager (Nicholas Bell)...
3,Negative,It must be assumed that those who praised this...
4,Positive,Superbly trashy and wondrously unpretentious 8...


In [4]:
train['review']=train['review'].str.replace("[^a-zA-Z]"," ")
train['sentiment']=train['sentiment'].replace("Positive",'1')
train['sentiment']=train['sentiment'].replace("Negative",'0')
train['sentiment']=train['sentiment'].astype('int32')
train['review']=train['review'].str.lower()
nltk.download('punkt')
train['review']=[nltk.word_tokenize(title) for title in train['review']]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\neera\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
train.head()

,sentiment,review
0,1,"[with, all, this, stuff, going, down, at, the,..."
1,1,"[the, classic, war, of, the, worlds, by, timot..."
2,0,"[the, film, starts, with, a, manager, nicholas..."
3,0,"[it, must, be, assumed, that, those, who, prai..."
4,1,"[superbly, trashy, and, wondrously, unpretenti..."


In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
max_fatures = 5000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(train['review'].values)
X = tokenizer.texts_to_sequences(train['review'].values)

In [7]:
X=pad_sequences(X, maxlen=250, dtype='int32', padding='post', truncating='post')
X  

array([[  16,   30,   10, ...,    3, 2303,    2],
       [   1,  348,  319, ...,    0,    0,    0],
       [   1,   19,  505, ...,   16, 1741,  380],
       ...,
       [ 224,    6,    3, ...,    0,    0,    0],
       [  10,  770,  642, ...,    0,    0,    0],
       [   9,  216,   10, ...,    0,    0,    0]])

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
embedding_vector_length = 100
model = Sequential() 
model.add(Embedding(5000, embedding_vector_length, input_length=250)) 
model.add(LSTM(100)) 
model.add(Dropout(0.2))
model.add(Dense(50, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary()) 





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 585,501
Trainable params: 585,501
Non-trainab

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,train['sentiment'], test_size = 0.2, random_state = 42)

In [14]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), nb_epoch=7, batch_size=128) 

C:\Users\neera\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 20000 samples, validate on 5000 samples
Epoch 1/7
20000/20000 [==============================] - 141s 7ms/step - loss: 0.5537 - acc: 0.7144 - val_loss: 0.5422 - val_acc: 0.7602
Epoch 2/7
20000/20000 [==============================] - 248s 12ms/step - loss: 0.4734 - acc: 0.8024 - val_loss: 0.4919 - val_acc: 0.7926
Epoch 3/7
20000/20000 [==============================] - 266s 13ms/step - loss: 0.4472 - acc: 0.8118 - val_loss: 0.4852 - val_acc: 0.7706
Epoch 4/7
20000/20000 [==============================] - 272s 14ms/step - loss: 0.4290 - acc: 0.8170 - val_loss: 0.4700 - val_acc: 0.8020
Epoch 5/7
20000/20000 [==============================] - 291s 15ms/step - loss: 0.5971 - acc: 0.6272 - val_loss: 0.6816 - val_acc: 0.5744
Epoch 6/7
20000/20000 [==============================] - 316s 16ms/step - loss: 0.4681 - acc: 0.7989 - val_loss: 0.4836 - val_acc: 0.8186
Epoch 7/7
20000/20000 [==============================] - 281s 14ms/step - loss: 0.3939 - acc: 0.8550 - val_loss: 0.4951 - va

In [15]:
score = model.evaluate(X_test, Y_test, verbose=1)

5000/5000 [==============================] - 36s 7ms/step


In [16]:
print(score[1])

0.8116
